In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("MovieLenssql")

config.set("spark.local.dir", "home/ubuntu/spark-temp")

config.set("hive.metastore.uris", "thrift://localhost:9083")
config.set("hive.metastore.warehouse.dir", "hdfs://localhost:9000/user/hive/warehouse")


from pyspark.sql import SparkSession
spark =SparkSession.builder\
                    .config(conf=config)\
                    .enableHiveSupport()\
                    .getOrCreate()

sc= spark.sparkContext

22/03/08 02:09:31 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/08 02:09:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/08 02:09:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/08 02:09:31 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/03/08 02:09:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/08 02:09:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/

In [3]:
spark.sql("""
SELECT * FROM moviedb.movies LIMIT 5
""").show()

+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|    null|               title|              genres|
|       1|    Toy Story (1995)|Adventure|Animati...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|Comedy|Drama|Romance|
+--------+--------------------+--------------------+



In [6]:
spark.sql("""
SELECT * FROM moviedb.ratings LIMIT 5
""").show()

+-------+--------+------+----------------+
|user_id|movie_id|rating|rating_timestamp|
+-------+--------+------+----------------+
|   null|    null|  null|            null|
|      1|       1|     4|       964982703|
|      1|       3|     4|       964981247|
|      1|       6|     4|       964982224|
|      1|      47|     5|       964983815|
+-------+--------+------+----------------+



In [7]:
popularMoviesDf = spark.sql("""
SELECT movie_id, avg(rating) as avg_rating, count(user_id) as total_ratings
FROM moviedb.ratings
WHERE movie_id IS NOT null
GROUP BY movie_id
""")

popularMoviesDf.printSchema()
popularMoviesDf.show(5)

root
 |-- movie_id: integer (nullable = true)
 |-- avg_rating: decimal(14,4) (nullable = true)
 |-- total_ratings: long (nullable = false)



[Stage 4:>                                                          (0 + 1) / 1]

+--------+----------+-------------+
|movie_id|avg_rating|total_ratings|
+--------+----------+-------------+
|    1580|    3.6364|          165|
|    2366|    3.7200|           25|
|    3175|    3.6933|           75|
|    1088|    3.5714|           42|
|   32460|    4.5000|            4|
+--------+----------+-------------+
only showing top 5 rows



In [8]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW popular_movies AS
SELECT movie_id, avg(rating) as avg_rating, count(user_id) as total_ratings
FROM moviedb.ratings
WHERE movie_id IS NOT null
GROUP BY movie_id
HAVING avg_rating >= 3.5 AND total_ratings >= 100
""")

DataFrame[]

In [ ]:
# for dropping a table
#spark.sql("DROP VIEW populat_movies")

In [9]:
spark.sql("SHOW TABLES").show()

+--------+--------------+-----------+
|database|     tableName|isTemporary|
+--------+--------------+-----------+
| default|        brands|      false|
| default|     employees|      false|
| default|        movies|      false|
| default|      payroles|      false|
|        |popular_movies|       true|
+--------+--------------+-----------+



In [10]:
spark.sql("SELECT * FROM popular_movies").show(5)

+--------+----------+-------------+
|movie_id|avg_rating|total_ratings|
+--------+----------+-------------+
|    1580|    3.6364|          165|
|    1721|    3.5571|          140|
|     858|    4.3958|          192|
|    1270|    4.1696|          171|
|    1265|    4.0699|          143|
+--------+----------+-------------+
only showing top 5 rows



In [11]:
spark.sql("""
SELECT movies.movie_id, title, avg_rating, total_ratings 
FROM popular_movies 
INNER JOIN moviedb.movies movies ON popular_movies.movie_id = movies.movie_id
ORDER BY avg_rating DESC
""").show()

22/03/08 02:31:17 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
[Stage 14:===================================================>  (189 + 1) / 200]

+--------+--------------------+----------+-------------+
|movie_id|               title|avg_rating|total_ratings|
+--------+--------------------+----------+-------------+
|     318|"Shawshank Redemp...|    4.5300|          317|
|   48516|           "Departed|    4.4673|          107|
|    2959|   Fight Club (1999)|    4.4404|          218|
|   58559|        "Dark Knight|    4.4295|          149|
|    1213|   Goodfellas (1990)|    4.3968|          126|
|     858|          "Godfather|    4.3958|          192|
|    2329|American History ...|    4.3876|          129|
|    1197|     "Princess Bride|    4.3732|          142|
|      50|     "Usual Suspects|    4.3725|          204|
|    1221| "Godfather: Part II|    4.3721|          129|
|     912|   Casablanca (1942)|    4.3600|          100|
|    7361|Eternal Sunshine ...|    4.3511|          131|
|     527|Schindler's List ...|    4.3500|          220|
|    1193|One Flew Over the...|    4.3308|          133|
|     260|Star Wars: Episod...|

In [12]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW most_popular_movies AS 
SELECT movies.movie_id, title, avg_rating, total_ratings 
FROM popular_movies 
INNER JOIN moviedb.movies movies ON popular_movies.movie_id = movies.movie_id
ORDER BY avg_rating DESC
""")

DataFrame[]

In [13]:
spark.sql("SELECT * FROM most_popular_movies").show(5)

22/03/08 02:31:56 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
[Stage 17:============================================>         (164 + 1) / 200]

+--------+--------------------+----------+-------------+
|movie_id|               title|avg_rating|total_ratings|
+--------+--------------------+----------+-------------+
|     318|"Shawshank Redemp...|    4.5300|          317|
|   48516|           "Departed|    4.4673|          107|
|    2959|   Fight Club (1999)|    4.4404|          218|
|   58559|        "Dark Knight|    4.4295|          149|
|    1213|   Goodfellas (1990)|    4.3968|          126|
+--------+--------------------+----------+-------------+
only showing top 5 rows



In [14]:
mostPopularMoviesDf = spark.table("most_popular_movies")

mostPopularMoviesDf.write\
                    .mode('overwrite')\
                    .saveAsTable("moviedb.most_popular_movies2")

mostPopularMoviesDf.show(5)

22/03/08 02:32:17 WARN LazyStruct: Extra bytes detected at the end of the row! Ignoring similar problems.
[Stage 23:>                                                       (2 + 1) / 121]22/03/08 02:32:21 WARN DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:609)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeInternal(DFSOutputStream.java:577)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:573)
[Stage 23:====>                                                  (10 + 1) / 121]22/03/08 02:32:22 WARN DFSClient: Caught exception 
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DFSO

+--------+--------------------+----------+-------------+
|movie_id|               title|avg_rating|total_ratings|
+--------+--------------------+----------+-------------+
|     318|"Shawshank Redemp...|    4.5300|          317|
|   48516|           "Departed|    4.4673|          107|
|    2959|   Fight Club (1999)|    4.4404|          218|
|   58559|        "Dark Knight|    4.4295|          149|
|    1213|   Goodfellas (1990)|    4.3968|          126|
+--------+--------------------+----------+-------------+
only showing top 5 rows



In [15]:
spark.sql("SELECT * FROM moviedb.most_popular_movies2").show(5)

+--------+--------------------+----------+-------------+
|movie_id|               title|avg_rating|total_ratings|
+--------+--------------------+----------+-------------+
|    4896|Harry Potter and ...|    3.9533|          107|
|    1198|Raiders of the Lo...|    4.3200|          200|
|     293|Léon: The Profess...|    4.1429|          133|
|    6539|Pirates of the Ca...|    3.9664|          149|
|     253|Interview with th...|    3.5596|          109|
+--------+--------------------+----------+-------------+
only showing top 5 rows



In [16]:
spark.sql("SHOW TABLES IN moviedb").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| moviedb|               links|      false|
| moviedb| most_popular_movies|      false|
| moviedb|most_popular_movies2|      false|
| moviedb|              movies|      false|
| moviedb|             ratings|      false|
| moviedb|             reviews|      false|
| moviedb|                tags|      false|
|        | most_popular_movies|       true|
|        |      popular_movies|       true|
+--------+--------------------+-----------+

